In [1]:
from lib.data import load_file

train = load_file()
chi2_df = load_file("chi2")

2017-10-21 13:44:45,483 - INFO - Using already cached file data/train.csv
2017-10-21 13:44:49,869 - INFO - Using already cached file data/chi2_features.csv


In [2]:
def n_best(chdf, n = 15):
    sorted = chdf.sort_values('chi2', axis = 0, ascending = False)
    return sorted['feature'][:n]

columns = list(n_best(chi2_df))

In [3]:
X = train[columns]
y = train.target

print(X.shape)
print(y.shape)

(595212, 15)
(595212,)


In [4]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

pipe = make_pipeline(StandardScaler(), LogisticRegression())
pipe.fit(X, y)

Pipeline(memory=None,
     steps=[('standardscaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('logisticregression', LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])

In [5]:
from sklearn.model_selection import cross_val_score, cross_val_predict

cross_val_score(pipe, X, y)

array([ 0.96354931,  0.96355416,  0.96355398])

In [6]:
from sklearn.metrics import classification_report

y_pred = pipe.predict(X)

In [7]:
test = load_file("test")
X_test = test[columns]

2017-10-21 13:44:57,196 - INFO - Using already cached file data/test.csv


In [8]:
y_test_pred = pipe.predict_proba(X_test)
y_test_pred[:10]
y_test_pred.shape

(892816, 2)

In [9]:
test['target'] = y_test_pred[:,1]
test.head()

,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_10_bin,...,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin,target
id,,,,,,,,,,,,,,,,,,,,,
0,0,1,8,1,0,0,1,0,0,0,...,1,1,12,0,1,1,0,0,1,0.026347
1,4,2,5,1,0,0,0,0,1,0,...,0,3,10,0,0,1,1,0,1,0.026720
2,5,1,3,0,0,0,0,0,1,0,...,0,2,4,0,0,0,0,0,0,0.028786
3,0,1,6,0,0,1,0,0,0,0,...,1,0,5,1,0,1,0,0,0,0.024158
4,5,1,7,0,0,0,0,0,1,0,...,0,0,4,0,1,1,0,0,1,0.033273


In [10]:
from datetime import datetime


ts = datetime.now().strftime("%Y.%m.%d.%H.%M.%s")
fname = "data/submission-mvp-{}.csv".format(ts)
print("Writing {}".format(fname))
test.to_csv(fname, columns = ['target'])

Writing data/submission-mvp-2017.10.21.13.45.1508607904.csv


In [11]:
import gzip

help(gzip.open)

Help on function open in module gzip:

open(filename, mode='rb', compresslevel=9, encoding=None, errors=None, newline=None)
    Open a gzip-compressed file in binary or text mode.
    
    The filename argument can be an actual filename (a str or bytes object), or
    an existing file object to read from or write to.
    
    The mode argument can be "r", "rb", "w", "wb", "x", "xb", "a" or "ab" for
    binary mode, or "rt", "wt", "xt" or "at" for text mode. The default mode is
    "rb", and the default compresslevel is 9.
    
    For binary mode, this function is equivalent to the GzipFile constructor:
    GzipFile(filename, mode, compresslevel). In this case, the encoding, errors
    and newline arguments must not be provided.
    
    For text mode, a GzipFile object is created, and wrapped in an
    io.TextIOWrapper instance with the specified encoding, error handling
    behavior, and line ending(s).



In [12]:
from lib.submit import write_submission_file

write_submission_file(test, columns = ['target'], name = 'mvp')

2017-10-21 13:45:16,576 - INFO - Created submission file data/submissions/mvp-2017.10.21.13.45.1508607906.csv.gz
